<center><h1>TP3 RCR2:</h1></center>
<center><h2>Réseaux Bayésiens</h2></center>
<center><h3>Conception d’un arbre multi-connected</h3></center>

In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork

#Etablissement de la structure
weather_model = BayesianNetwork([
    ('F','N'),
    ('F','Ne'),
    ('N','Ne'),
    ('Ne','B'),
    ('R','B'),
    ])

#Définition des relations
froid_cpd= TabularCPD(
    variable='F',
    variable_card= 2,
    values=[[.75],[.25]]
)

nuage_cpd= TabularCPD(
    variable='N',
    variable_card=2,
    values=[[.6,.3],
    [.4,.7]],
    evidence=['F'],
    evidence_card=[2]
)

neige_cpd= TabularCPD(
    variable='Ne',
    variable_card=2,
    values=[
    [.99,.8,.7,.4], [.01,.2,.3,.6]],
    evidence=['F','N'],
    evidence_card=[2,2]
)

route_frequente_cpd= TabularCPD(
    variable='R',
    variable_card=2,
    values=[[.6],[.4]]
)

route_bloque_cpd= TabularCPD(
    variable='B',
    variable_card=2,
    values=[[.9,.4,.2,.3], [.1,.6,.8,.7]],
    evidence=['R','Ne'],
    evidence_card=[2,2]
)

#on ajoute les relation à notre model
weather_model.add_cpds(froid_cpd,
nuage_cpd,neige_cpd,route_bloque_cpd,route_frequente_cpd)
#reverifier la structure de notre model
weather_model.get_cpds()

[<TabularCPD representing P(F:2) at 0x1b40d7cb220>,
 <TabularCPD representing P(N:2 | F:2) at 0x1b40d76fe80>,
 <TabularCPD representing P(Ne:2 | F:2, N:2) at 0x1b40d7cb310>,
 <TabularCPD representing P(B:2 | R:2, Ne:2) at 0x1b40d7cb250>,
 <TabularCPD representing P(R:2) at 0x1b40d7cb280>]

In [3]:
#trouver les chemins des noeuds pour determiner qu'est ce qui donne des infos sur un noeud
print(weather_model.active_trail_nodes('F'))
print()

print(weather_model.active_trail_nodes('B'))
print()

print(weather_model.active_trail_nodes('Ne'))

{'F': {'F', 'B', 'Ne', 'N'}}

{'B': {'R', 'F', 'N', 'Ne', 'B'}}

{'Ne': {'B', 'F', 'Ne', 'N'}}


In [4]:
#determinaison des independances locales
print(weather_model.local_independencies('F'))
print()

print(weather_model.local_independencies('Ne'))
print()

print(weather_model.local_independencies('B'))
print()

weather_model.get_independencies()

(F ⟂ R)

(Ne ⟂ R | F, N)

(B ⟂ F, N | R, Ne)



(F ⟂ R)
(F ⟂ R, B | Ne)
(F ⟂ R | N)
(F ⟂ B | R, Ne)
(F ⟂ R | B, Ne)
(F ⟂ R, B | Ne, N)
(F ⟂ B | R, Ne, N)
(F ⟂ R | B, Ne, N)
(R ⟂ F, Ne, N)
(R ⟂ Ne, N | F)
(R ⟂ F, N | Ne)
(R ⟂ F, Ne | N)
(R ⟂ N | F, Ne)
(R ⟂ Ne | F, N)
(R ⟂ F, N | B, Ne)
(R ⟂ F | Ne, N)
(R ⟂ N | F, B, Ne)
(R ⟂ F | B, Ne, N)
(Ne ⟂ R)
(Ne ⟂ R | F)
(Ne ⟂ R | N)
(Ne ⟂ R | F, N)
(N ⟂ R)
(N ⟂ R | F)
(N ⟂ B, R | Ne)
(N ⟂ B, R | F, Ne)
(N ⟂ R | B, Ne)
(N ⟂ B | R, Ne)
(N ⟂ R | F, B, Ne)
(N ⟂ B | F, Ne, R)
(B ⟂ F, N | Ne)
(B ⟂ N | F, Ne)
(B ⟂ F | Ne, N)
(B ⟂ F, N | R, Ne)
(B ⟂ N | F, Ne, R)
(B ⟂ F | R, Ne, N)

In [5]:
from pgmpy.inference import VariableElimination
weather_infer = VariableElimination(weather_model)
#calcul des proba existante pour vérifier que tout est correct
prob_froid = weather_infer.query(variables=['F'])
print(prob_froid)

+------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.7500 |
+------+----------+
| F(1) |   0.2500 |
+------+----------+


In [6]:
#faire un autre calcul de proba
prob_bloque= weather_infer.query(variables=['B'])
print(prob_bloque)

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.5701 |
+------+----------+
| B(1) |   0.4299 |
+------+----------+


In [8]:
#probabilité d'avoir une route bloqué quand il fait froid (proba conditionelle)
proba_bloque_froid = weather_infer.query(variables=['B'], evidence= {'F':1}) 
print(proba_bloque_froid)

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.4874 |
+------+----------+
| B(1) |   0.5126 |
+------+----------+


In [7]:
#probabilité d'avoir de la neige quand il fait pas froid mais quand il y a des nuages
proba_neige_nonFroidetNuage = weather_infer.query(variables=['Ne'], evidence= {'F':0, 'N':1}) 
print(proba_neige_nonFroidetNuage)

+-------+-----------+
| Ne    |   phi(Ne) |
+=======+===========+
| Ne(0) |    0.8000 |
+-------+-----------+
| Ne(1) |    0.2000 |
+-------+-----------+
